*Role Title and Role Code are similar based on pandas
*Instead of using hashing, can consider cantor(m,n) = 1/2 * (m+n)(m+n+1)+m. Beware of hash collision
*PCA / SVD are more suited for continuous values. For discrete variables, consider using MCA
1. Feature Extraction - Using Genetic Algorithm / Greedy Algorithm [sklearn.feature_selection.RFE, RFECV , Gradient Boosting
2. Try using Stacked Classifier 
  * np.mean
  * weighted mean
  * jaccard score
  * pass the results from each classifer into another RF / Linear Regression
3. Data Exploratory - Find if train has more classes compared to test ? 

In [2]:
import pandas as pd
from pivottablejs import pivot_ui
import seaborn as sb
import matplotlib.pyplot as plt  
from ipywidgets import widgets
import numpy as np
from sklearn import (metrics, cross_validation, linear_model, preprocessing)
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_selection import RFE
from sklearn import ensemble
from itertools import combinations

%matplotlib inline

sb.set(style="ticks", color_codes=True)
df = pd.read_csv('train.csv')
dft = pd.read_csv('test.csv')
#g = sb.pairplot(df)

In [ ]:
#Observation Only
df.head()

In [3]:
#len(df['RESOURCE'].value_counts())
#df['RESOURCE'].plot.hist(bins=7518)

#dfc = df.iloc[:,1:10] 
dfc = df.iloc[:,:]
for col in df.columns:
  dfc[col] = df[col].astype('category')

dfc.describe()


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
count,32769,32769,32769,32769,32769,32769,32769,32769,32769,32769
unique,2,7518,4243,128,177,449,343,2358,67,343
top,1,4675,770,117961,118300,117878,118321,117906,290919,118322
freq,30872,839,152,21407,4424,1135,4649,6896,10980,4649


In [ ]:
#Observation Only
#test = pd.pivot_table(dfc,index=['ROLE_TITLE','ROLE_CODE'],aggfunc='count',dropna=True)
#dfc.pivot_table(index='ROLE_TITLE',aggfunc=lambda x:len(x.unique()))
mult = {}
for c in df.columns:
    temp = df.pivot_table(index=c,aggfunc=lambda x:len(x.unique()),fill_value=0).apply(np.max)
    mult[c] = temp

multpd = pd.DataFrame(mult)


In [4]:
def save_results(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))


In [26]:
def cantor(pairs):
    result2 = 0.5 * (pairs[0] + pairs[1]) * (pairs[0] + pairs[1] +1) + pairs[1]
    if(len(pairs) == 2): 
        return result2
    if(len(pairs) == 3 ):
        result3 = 0.5 * (result2 + pairs[2]) * (result2 + pairs[2] +1) + pairs[2]
        return result3
    

In [27]:
# Convert from dataframe to np.array
y = np.ravel(df.iloc[:,:1])
X = np.array(df.iloc[:,1:9])
X_test = np.array(dft.iloc[:,1:9])
header = list(df.columns[1:])

data = np.vstack((X,X_test))

In [28]:
# yc = combinations(range(n),2)
# to view all the combinations
# list(yc) 

#train data[:32769,:]
#test data[32769:,:]
#tuple is used here so that the result is hashable

test_offset = 32769
# Get Combinations of 2 features 
for indices in combinations(range(8),2):
    #data = np.column_stack((data,list(ctypes.c_size_t(hash(tuple(v))).value for v in data[:,indices]))) 
    #data = np.column_stack((data,list(np.uint64(hash(tuple(v))) for v in data[:,indices]))) 
    #data = np.column_stack((data,list(hash(tuple(v)) for v in data[:,indices]))) 
    data = np.column_stack((data,list(cantor(tuple(v)) for v in data[:,indices]))) 
    #a,b = indices
    #header.append(header[a] +"&"+ header[b])
print('2 Combinations')
print(data.shape)
print(np.any(data < 0))
print(np.ndarray.min(data))
print(np.ndarray.max(data))
    
# Get Combinations of 3 features 
for indices in combinations(range(8),3):
    data = np.column_stack((data,list(cantor(tuple(v)) for v in data[:,indices]))) 
    
#Check loaded correctly with non-zero elements    
print('3 Combinations')
print(data.shape)
print(np.any(data < 0))
print(np.ndarray.min(data))
print(np.ndarray.max(data))

2 Combinations
(91690, 36)
False
0
194537644783.0
3 Combinations
(91690, 92)
False
0
1.892247063879666e+22


In [29]:


# Simplify the encoding. Probably not essential. Could possibly improve performance
# Vertical Stacking is required so that all the features are captured. There might be some features in test set not in train set

le = preprocessing.LabelEncoder()
data_e = []
# To perform column wise operations, use Transpose function
for i in data.T:
    le.fit(i)
    data_e.append(le.transform(i))

data = np.asarray(data_e).T

print(np.any(data < 0))
print(np.ndarray.min(data))
print(np.ndarray.max(data))



False
0
81232


In [ ]:
# #Code to help Visualize what the algorithm is doing 
# t_data = []
# test_data = np.array([[1,2,3,4],[1,2,2,0],[1,2,3,4],[1,4,5,6],[2,3,4,5]])
# print(test_data.shape)

# #if the combination of categories are the same, they will be hashed to the same value. Eg 1 row with (0,1,0) will have same 
# #hash value of another row with (0,1,0)
# for indices in combinations(range(4),2):
#     t_data.append(hash(tuple(v)) for v in test_data[:,indices])
#     #print(v for v in test_data[:,indices])
#     print(indices)
    
# #Comparison 1 - Combinations are (0,1),(0,2),(0,3),(1,2),(1,3),(2,3)
# # for v in test_data[:,(0,1)]:
# #   print(hash(tuple(v)))

# #Comparison 2 - t_data[0] corresponds to (0,1)
# #tuple(t_data[0])

In [30]:
encoder = preprocessing.OneHotEncoder(dtype=np.uint32)
# we want to encode the category IDs encountered both in
# the training and the test set, so we fit the encoder on both
encoder.fit(data)
X = encoder.transform(data[:test_offset,:])
X_test = encoder.transform(data[test_offset:,:])

# encoder = preprocessing.OneHotEncoder()
# encoder.fit(data)
# X = encoder.transform(X)
# X_test = encoder.transform(X_test)

In [31]:
SEED = 42
mean_auc = 0.0

n = 10  # repeat the CV procedure 10 times to get more precise results
model = linear_model.LogisticRegression(C=1)
#model = BernoulliNB(alpha=0.005)
for i in range(n):
    # for each iteration, randomly hold out 20% of the data as CV set
    X_train, X_cv, y_train, y_cv = cross_validation.train_test_split(X, y, test_size=.20, random_state=i*SEED)

    # if you want to perform feature selection / hyperparameter
    # optimization, this is where you want to do it

    # train model and make predictions
    model.fit(X_train, y_train) 
    preds = model.predict_proba(X_cv)[:, 1]

    # compute AUC metric for this CV fold
    fpr, tpr, thresholds = metrics.roc_curve(y_cv, preds)
    roc_auc = metrics.auc(fpr, tpr)
    print("AUC (fold %d/%d): %f" % (i + 1, n, roc_auc))
    mean_auc += roc_auc

print("Mean AUC: %f" % (mean_auc/n))

# === Predictions === #
# When making predictions, retrain the model on the whole training set
model.fit(X, y)
preds = model.predict_proba(X_test)[:, 1]
save_results(preds, "submit.csv")

AUC (fold 1/10): 0.883359
AUC (fold 2/10): 0.881408
AUC (fold 3/10): 0.891638
AUC (fold 4/10): 0.899944
AUC (fold 5/10): 0.883712
AUC (fold 6/10): 0.889287
AUC (fold 7/10): 0.874977
AUC (fold 8/10): 0.884113
AUC (fold 9/10): 0.882565
AUC (fold 10/10): 0.892604
Mean AUC: 0.886361


In [ ]:
estimator = linear_model.LogisticRegression(C=1)
selector = RFE(estimator)
selector = selector.fit(X, y)
selector.ranking_

In [ ]:
# Numpy Operations
# print(np.fromiter(t_data[0],np.int32))
# print(np.fromiter(t_data[1],np.int32))
# print(np.fromiter(t_data[2],np.int32))
# print(np.fromiter(t_data[3],np.int32))
# print(np.fromiter(t_data[4],np.int32))
# print(np.fromiter(t_data[5],np.int32))

# Convert to numpy array - Use np.array(list)
# for i in nparray - iterate through rows
# for i in nparray.T - iterate through columns
# Initialize np.zeros(5)

# To view generator operators (Use list,tuples)
# for i,j in enumerate(t_data):
#     #lst_array[i] = np.array(list(j))
#     lst_array[i] = np.fromiter(j,np.float)

AUC = Logistic Regression returns the probability that a certain input features would be in a certain class
ROC curve visualizaes all possible thresholds
Misclassification rate is error rate for a single threshold


Attempt with original features : Public : 0.88515, Private : 0.88205
Attempt with original features + combination of 2 : Public : 0.90141, Private 0.89582 